# Matrice en 3 colonnes

In [ ]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

Ce notebook propose d'implémenter un produit matriciel sous Spark. Spark comme SQL n'aime pas trop avoir un nombre de colonnes variables. La première étape consiste à transformer les matrices $I\times J$ en tableau de trois colonnes $(i,j,coefficient)$.

## Création d'une matrice aléatoire

In [ ]:
from numpy.random import rand
rnd1 = rand(10,10)
rnd2 = rand(10, 2)
rnd1 @ rnd2

array([[ 2.49193185,  1.93124783],
       [ 3.67023704,  3.16655504],
       [ 2.37311039,  2.38221324],
       [ 2.25001061,  2.60441484],
       [ 2.79618265,  2.98784145],
       [ 2.4902488 ,  1.72328881],
       [ 2.61943371,  2.22717208],
       [ 2.83936229,  2.9615508 ],
       [ 2.3475777 ,  2.24761376],
       [ 3.02640224,  2.27159035]])

In [ ]:
import pandas
df1 = pandas.DataFrame(rnd1)
df2 = pandas.DataFrame(rnd2)
df2

,0,1
0,0.825367,0.104939
1,0.257015,0.241694
2,0.039147,0.789518
3,0.491701,0.153167
4,0.872270,0.309497
5,0.786646,0.685438
6,0.453832,0.876692
7,0.862785,0.496380
8,0.624780,0.481755
9,0.570678,0.779402


In [ ]:
df1.to_csv("rnd1.txt", sep="\t", header=None, index=False)
df2.to_csv("rnd2.txt", sep="\t", header=None, index=False)

## Conversion d'une matrice au format Spark

Lorsqu'un traitement est distribué en Map/Reduce, il n'est pas possible de s'appuyer sur l'ordre dans lequel sont traitées les lignes. Le plus est d'ajouter cette information sur chaque ligne plutôt que de chercher à la récupérer.

In [ ]:
df1.to_csv("rnd1.txt", sep="\t", header=None, index=True)
df2.to_csv("rnd2.txt", sep="\t", header=None, index=True)

In [ ]:
def process_mat_row(row):
    values = row.split("\t")
    index = int(values[0])
    values = [float(_) for _ in values[1:]]
    return [[index, j, v] for j, v in enumerate(values)]

In [ ]:
mat1 = sc.textFile("rnd1.txt")
new_mat1 = mat1.flatMap(process_mat_row)
new_mat1.take(12)

[[0, 0, 0.7073669487056997],
 [0, 1, 0.38049053626197293],
 [0, 2, 0.05182708138929548],
 [0, 3, 0.3599397323172999],
 [0, 4, 0.17035816004085025],
 [0, 5, 0.7665995035596439],
 [0, 6, 0.5334680552689206],
 [0, 7, 0.12390850009936782],
 [0, 8, 0.438956360362891],
 [0, 9, 0.44926935830522163],
 [1, 0, 0.5458627444692511],
 [1, 1, 0.05686919336554841]]

In [ ]:
mat2 = sc.textFile("rnd2.txt")
new_mat2 = mat2.flatMap(process_mat_row)
new_mat2.take(12)

[[0, 0, 0.8253673414743797],
 [0, 1, 0.10493947274099424],
 [1, 0, 0.25701493053720903],
 [1, 1, 0.24169374169340407],
 [2, 0, 0.039147207879637436],
 [2, 1, 0.7895175542881998],
 [3, 0, 0.4917011886170065],
 [3, 1, 0.15316743917896902],
 [4, 0, 0.8722700195595767],
 [4, 1, 0.30949671566398185],
 [5, 0, 0.7866459425892333],
 [5, 1, 0.6854376344061859]]

## Produit matriciel

Il faut d'abord faire la jointure avec la méthode [join](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.join). Il faut que la clé soit sur la première colonne.

In [ ]:
def key_ij(row):
    return row[0], (row[1], row[2])
def key_ji(row):
    return row[1], (row[0], row[2])
mat_join = new_mat1.map(key_ji).join(new_mat2.map(key_ij))
mat_join.take(12)

[(0, ((0, 0.7073669487056997), (0, 0.8253673414743797))),
 (0, ((0, 0.7073669487056997), (1, 0.10493947274099424))),
 (0, ((1, 0.5458627444692511), (0, 0.8253673414743797))),
 (0, ((1, 0.5458627444692511), (1, 0.10493947274099424))),
 (0, ((2, 0.7991774982704423), (0, 0.8253673414743797))),
 (0, ((2, 0.7991774982704423), (1, 0.10493947274099424))),
 (0, ((3, 0.786288741719609), (0, 0.8253673414743797))),
 (0, ((3, 0.786288741719609), (1, 0.10493947274099424))),
 (0, ((4, 0.6679236875594428), (0, 0.8253673414743797))),
 (0, ((4, 0.6679236875594428), (1, 0.10493947274099424))),
 (0, ((5, 0.595595156523955), (0, 0.8253673414743797))),
 (0, ((5, 0.595595156523955), (1, 0.10493947274099424)))]

On effectue le produit matriciel.

In [ ]:
def produit_matriciel(row):
    index, ((i, v1), (j, v2)) = row
    return i, j, v1 * v2
produit = mat_join.map(produit_matriciel)
produit.take(12)

[(0, 0, 0.5838375779000673),
 (0, 1, 0.07423071463158205),
 (1, 0, 0.45053728221249445),
 (1, 1, 0.057282548593555284),
 (2, 0, 0.6596150071136206),
 (2, 1, 0.08386526529496705),
 (3, 0, 0.6489770483843489),
 (3, 1, 0.08251272597823557),
 (4, 0, 0.5512823983087015),
 (4, 1, 0.0700915596037085),
 (5, 0, 0.4915847909351938),
 (5, 1, 0.06250144169271378)]

Il ne reste plus qu'à agréger [reduceByKey](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.reduceByKey). La documentation fournit un exemple facilement transposable. Elle indique aussi : *Merge the values for each key using an associative and commutative reduce function.* Pourquoi précise-t-elle **associative et commutative** ? Cela signifie que le résultat ne dépend pas de l'ordre dans lequel l'agrégation est réalisée et qu'on peut commencer à agréger sans attendre d'avoir regroupé toutes les valeurs associées à une clé.

* *Cas 1 :* [groupBy](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.groupBy) + agrégation qui commence une fois les valeurs regroupées
* *Cas 2 :* [reduceByKey](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.reduceByKey) + agrégation qui commence dès les premières valeurs regroupées

Le cas 2 est moins consommateur en terme de données. Le cas 1 n'est possible que si les valeurs agrégées ne sont pas trop nombreuses. Ca tombe bien, dans notre cas, le cas 2 convient.

In [ ]:
from operator import add
final = produit.map(lambda row: ((row[0], row[1]), row[2])).reduceByKey(add)
aslist = final.collect()
aslist.sort()
aslist

[((0, 0), 2.4919318533320625),
 ((0, 1), 1.9312478315672568),
 ((1, 0), 3.670237041775416),
 ((1, 1), 3.166555036937517),
 ((2, 0), 2.373110393276501),
 ((2, 1), 2.3822132377173104),
 ((3, 0), 2.2500106053379922),
 ((3, 1), 2.6044148377063303),
 ((4, 0), 2.7961826479381813),
 ((4, 1), 2.9878414514656617),
 ((5, 0), 2.490248798137291),
 ((5, 1), 1.7232888136146995),
 ((6, 0), 2.619433710358372),
 ((6, 1), 2.227172080670293),
 ((7, 0), 2.8393622934115),
 ((7, 1), 2.961550804570348),
 ((8, 0), 2.3475776976381812),
 ((8, 1), 2.247613763833498),
 ((9, 0), 3.0264022424819412),
 ((9, 1), 2.271590353117718)]

Résultat initial :

In [ ]:
rnd1 @ rnd2

array([[ 2.49193185,  1.93124783],
       [ 3.67023704,  3.16655504],
       [ 2.37311039,  2.38221324],
       [ 2.25001061,  2.60441484],
       [ 2.79618265,  2.98784145],
       [ 2.4902488 ,  1.72328881],
       [ 2.61943371,  2.22717208],
       [ 2.83936229,  2.9615508 ],
       [ 2.3475777 ,  2.24761376],
       [ 3.02640224,  2.27159035]])